<a href="https://colab.research.google.com/github/crdguez/mat1bac_cit/blob/master/notebooks/Ejercicios1_3.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

In [1]:
from sympy import *

init_session()

a, b, c, d = symbols('a b c d', real = True)


IPython console for SymPy 1.8 (Python 3.6.13-64-bit) (ground types: gmpy)

These commands were executed:
>>> from __future__ import division
>>> from sympy import *
>>> x, y, z, t = symbols('x y z t')
>>> k, m, n = symbols('k m n', integer=True)
>>> f, g, h = symbols('f g h', cls=Function)
>>> init_printing()

Documentation can be found at https://docs.sympy.org/1.8/



In [2]:
def sistema_a_latex(sist) :
    """
    Convierte un sistema Sympy a código LaTeX
    """
    sist_latex = r"$\left\{ \begin{matrix}"
    for e in sist:
        sist_latex += latex(e)+r" \\ "  
    sist_latex += r"\end{matrix}\right.$"
    return(sist_latex)



def enunciado_dicusion(sist, k=k,lista=[]) :
    """
    Típico enunciado de discusión y resolución de un sistema con un parámetro
    """
    sistema_problema = sist
    texto_problema = r"""Considere el siguiente sistema de ecuaciones, dónde {}  es un parámetro real: $""".format(k)
    texto_problema += sistema_a_latex(sistema_problema)
    texto_problema += """$ Determine los valores del parámetro real {}, para los que este sistema es compatible determinado,  compatible indeterminado o incompatible.""".format(k)
    
    if lista :
        texto_problema +=r" \\ Resuelva el sistema cuando"
        texto_problema +=r"\begin{itemize}"
        for i in lista :
            texto_problema += r"\item {}={}".format(k,i)        
        texto_problema +=r"\end{itemize}"
    return(texto_problema)


def discusion_solucion(sist, k=k, resol=True) :
    """
    Discute y resuelve un sistema con un parámetro o sin él
    """
    
    A, b = linear_eq_to_matrix(sist,[x,y,z])
    AA = A.row_join(b)
    AAs = AA.LUdecomposition()[1].applyfunc(nsimplify)
    
    if A.rank() < AA.rank():
        solucion = r"Como $rg(A)={} < rg(A^*)={} \to$ S.I. \\ ".format(A.rank(),AA.rank())
        solucion += r"Ya que escalonando la matriz ampliada: \\ $A^*= {} \thicksim {}$ \\".format(latex(AA),latex(AAs)).replace('[','(').replace(']',')')
        solucion += r"${}={} \quad y \quad {}={}$".format(latex(AAs[:,:-1]),latex(A.det()),latex(AAs[:,1:]),latex(AAs[:,1:].det())).replace('[','|').replace(']','|')

    else :  
        pprint("Discusión y resolución por Gauss:")

        solucion = r"\textbf{Discusión y resolución por Gauss:} Escalonando la matriz ampliada tenemos\\"
        solucion += r"$A^*= {} \thicksim {}$. \\  De los valores de la última fila podemos concluir:".format(latex(AA),latex(AAs)).replace('[','(').replace(']',')')
        solucion += r"\begin{itemize}"
        pprint(AAs)
        pprint("det(A)={}".format(A.det()))
        pprint(AAs.row(-1)[-2:])

        for i in (solve(AAs.row(-1)[-2])) :
            pprint(i)
            if AAs.row(-1)[-1].subs(k,i) == 0 :
                pprint("Si {} = {} -->  0z=0 --> S.C.I".format(k,i))
                solucion += r"\item Si ${} = {} \to$ $${}$$ La última fila es $0z=0 \to $ S.C.I".format(k,i,latex(AAs.subs(k,i))).replace('[','(').replace(']',')')
                pprint([eq.subs(k,i) for eq in sist])
                sol = list(zip([x,y,z],linsolve([eq.subs(k,i) for eq in sist],[x,y,z]).args[0],[r.subs(k,i) for r in [AAs.row(j) for j in range(AA.shape[0])]]))
                pprint(sol)
                for s in reversed(sol) :
                        pprint("{} --> {} = {}".format(s[2],s[0],s[1].subs(z,"\lambda")))
                        if resol :
                            solucion += r"\begin{itemize}"
                            solucion += r"\item ${} \to {} = {}$".format(latex(s[2]),latex(s[0]),latex(s[1]).replace('z',"\lambda")).replace('[','(').replace(']',')')
                            solucion += r"\end{itemize}"
            else :
                pprint("Si {} = {} --> 0z={} -->S.I".format(k,i, AAs.row(-1)[-1].subs(k,i)))
                solucion += r"\item Si ${} = {} \to$ $${}$$ La última fila es $0z={} \to $ S.I.".format(k,i,latex(AAs.subs(k,i)),AA.LUdecomposition()[1].applyfunc(simplify).row(-1)[-1].subs(k,i)).replace('[','(').replace(']',')')

        if solve(AAs.row(-1)[-2]) :
            pprint("si {} <> {}  --> S.C.D.".format(k, solve(AAs.row(-1)[-2])))
            solucion += r"\item si ${}\neq {}  \to $ S.C.D.".format(k,solve(AAs.row(-1)[-2]))
        else :
            pprint("S.C.D.".format(k, solve(AAs.row(-1)[-2])))
            solucion += r"\item S.C.D.".format(k,solve(AAs.row(-1)[-2]))


        pprint(list(linsolve(sist,[x,y,z]).args[0].args))
        sol = list(zip([x,y,z],linsolve(sist,[x,y,z]).args[0],[AAs.row(j) for j in range(AA.shape[0])]))
        pprint(sol)
        if resol :
            for s in reversed(sol) :
                pprint("{} --> {} = {}".format(s[2],s[0],s[1]))
                solucion += r"\begin{itemize}"
                solucion += r"\item ${} \to {} = {}$".format(latex(s[2]),s[0],latex(s[1])).replace('[','(').replace(']',')')
                solucion += r"\end{itemize}"

        solucion += r"\end{itemize}  "

        # Por rangos y determinantes

        pprint("Por rangos y determinantes:")
        solucion += r"\textbf{Por rangos y determinantes:} \\"


        #Rango de A:


        solucion += r"$\left|A\right|={}={} ".format(latex(A).replace("[","|").replace("]","|"),latex(A.det())) 
        if solve(A.det()) :
            solucion += r"\to \left|A\right|=0 \quad si \quad {} = {}$".format(k,latex(solve(A.det())))
            solucion += r"\begin{itemize}"
            for i in solve(A.det()):
                A2 = A.subs(k,i)
                AA2 = AA.subs(k,i)
                pprint(A2)
                pprint(A2.rank())
                pprint(AA2.rank())
                if A2.rank() < AA2.rank() :
                    pprint("Si {} = {} --> S.I.".format(k,i))
                    solucion += r"\item Si ${}={} \to rg(A)={} \land rg(A^*)={} \to $ S.I.".format(k,i,A2.rank(),AA2.rank())
                else :
                    pprint("Si {} = {} --> S.C.I --> solo se puede resolver por Gauss, ver más arriba".format(k,i))
                    solucion += r"\item Si ${}={} \to rg(A)={} \land rg(A^*)={} \to $ S.C.I. $\to$ solo se puede resolver por Gauss, (ver más arriba)".format(k,i,A2.rank(),AA2.rank())

            pprint("Si {} <> {} --> Rango(A)=Rango(A)={} --> S.C.D.".format(k,solve(A.det()),A.rank()))
            solucion += r"\item Si ${} \neq{} \to rg(A)={} \land rg(A^*)={} \to $ S.C.D.  \\ ".format(k, solve(A.det()),A.rank(),AA.rank())
            if resol :
                pprint("Por Cramer:")
                sol=[]
                solucion += r" \\ Por Cramer: \begin{itemize}"
                for i, var in enumerate([x,y,z]):
                    #print("columna"+latex(i))
                    AA.col_swap(i,3)
                    pprint("Delta_i, A.det, solucion_i : ")
                    sol.append([var,AA[:,:-1],simplify(AA[:,:-1].det()),simplify(A.det()),simplify(AA[:,:-1].det()/A.det())])
                    pprint(sol[i])
                    #print(r"$x_"+latex(i)+r"=\frac{"+latex(AA[:,:-1])+r"}{"+latex(A.det())+r"}=\frac{"+latex(AA[:,:-1].det())+r"}{"+latex(A.det())+r"}="+latex(AA[:,:-1].det()/A.det())+"$")
                    solucion += r"\item $"+latex(sol[i][0])+r"=\frac{"+latex(sol[i][1]).replace('[','|').replace(']','|')+r"}{"+latex(sol[i][3])+r"}=\frac{"+latex(sol[i][2])+r"}{"+latex(sol[i][3])+r"}="+latex(sol[i][4])+r"$"
                    AA.col_swap(i,3)
                solucion += r"\end{itemize}"

        else:
            solucion += r" \neq 0 $"
            solucion += r"\begin{itemize}"
            solucion += r" \item $rg(A)={} \land rg(A^*)={} \to $ S.C.D.   \\".format(A.rank(),AA.rank())
            if resol :
                pprint("Por Cramer:")
                sol=[]
                solucion += r" \\ Por Cramer: \begin{itemize}"
                for i, var in enumerate([x,y,z]):
                    #print("columna"+latex(i))
                    AA.col_swap(i,3)
                    pprint("Delta_i, A.det, solucion_i : ")
                    sol.append([var,AA[:,:-1],simplify(AA[:,:-1].det()),simplify(A.det()),simplify(AA[:,:-1].det()/A.det())])
                    pprint(sol[i])
                    #print(r"$x_"+latex(i)+r"=\frac{"+latex(AA[:,:-1])+r"}{"+latex(A.det())+r"}=\frac{"+latex(AA[:,:-1].det())+r"}{"+latex(A.det())+r"}="+latex(AA[:,:-1].det()/A.det())+"$")
                    solucion += r"\item $"+latex(sol[i][0])+r"=\frac{"+latex(sol[i][1]).replace('[','|').replace(']','|')+r"}{"+latex(sol[i][3])+r"}=\frac{"+latex(sol[i][2])+r"}{"+latex(sol[i][3])+r"}="+latex(sol[i][4])+r"$"
                    AA.col_swap(i,3)
                solucion += r"\end{itemize}"
        solucion += r"\end{itemize}"
    return(solucion) 



In [3]:
A=Matrix(3,4,[2,0,a,2,-1,0,-1,3,5,a+4,-4,3])
AA= A
AAs = AA.LUdecomposition()[1].applyfunc(nsimplify)
A, AAs, AAs.subs(a,1).rank()
B=Matrix(3,3,[2,a,2,-1,-1,3,5,-4,3])
B, B.det(), B.det().subs(a,-4), B.det().subs(a,2)


⎛⎡2   a   2⎤                    ⎞
⎜⎢         ⎥                    ⎟
⎜⎢-1  -1  3⎥, 18⋅a + 36, -36, 72⎟
⎜⎢         ⎥                    ⎟
⎝⎣5   -4  3⎦                    ⎠

In [3]:
# Típico ejercicio que hay que discutir un sistema según el parámetro y resolverlo para algún valor del 
# parámetro k

sist = [Eq(2*x-y+k*z,1),Eq(-x+y-k*z,0),Eq(2*x-k*y+2*k*z,-1)]
lista =  [1]


def evau_sistema_param(sist,lista) :
    enunciado_latex = enunciado_dicusion(sist, lista=lista)
    solucion_latex = discusion_solucion(sist, resol=False)
    for i in lista :
        solucion_latex += r"\textbf{Para k = "+latex(i)+r" :}  \\ "
        solucion_latex += discusion_solucion([j.subs(k,i) for j in sist])
    return enunciado_latex, solucion_latex

enunciado_latex, solucion_latex = evau_sistema_param(sist, lista)
print(enunciado_latex+solucion_latex)


Discusión y resolución por Gauss:
⎡2  -1          k           1  ⎤
⎢                              ⎥
⎢              -k              ⎥
⎢0  1/2        ───         1/2 ⎥
⎢               2              ⎥
⎢                              ⎥
⎢        k⋅(2 - 2⋅k)           ⎥
⎢0   0   ─────────── + k  k - 3⎥
⎣             2                ⎦
det(A)=-k**2 + 2*k
⎡k⋅(2 - 2⋅k)           ⎤
⎢─────────── + k, k - 3⎥
⎣     2                ⎦
0
Si k = 0 --> 0z=-3 -->S.I
2
Si k = 2 --> 0z=-1 -->S.I
si k <> [0, 2]  --> S.C.D.
⎡     1     3 - k  ⎤
⎢1, ─────, ────────⎥
⎢   k - 2   2      ⎥
⎣          k  - 2⋅k⎦
⎡                       ⎛     1    ⎡        -k      ⎤⎞  ⎛    3 - k    ⎡      k
⎢(x, 1, [2  -1  k  1]), ⎜y, ─────, ⎢0  1/2  ───  1/2⎥⎟, ⎜z, ────────, ⎢0  0  ─
⎢                       ⎝   k - 2  ⎣         2      ⎦⎠  ⎜    2        ⎣       
⎣                                                       ⎝   k  - 2⋅k          

⋅(2 - 2⋅k)           ⎤⎞⎤
────────── + k  k - 3⎥⎟⎥
    2                ⎦⎟⎥
                

In [5]:
#discusion_solucion([Eq(k*x+y,1),Eq(2*x+(k+1)*y,2),Eq(x+k*y,1)])
# Hoja de problemas
problemas =[
#             ["""Halle tres números sabiendo que el primero menos el segundo es igual a un quinto del tercero, 
# si al doble del primero le restamos 6 nos queda la suma del segundo y el tercero y, además, 
# el triple del segundo menos el doble del tercero es igual al primero menos 8""",
#              [Eq(5*x-5*y-z,0),Eq(2*x-y-z,6),Eq(x-3*y+2*z,8)]
#             ],
#             ["""Una tienda vende una clase de calcetines a 1200 pts el par.  
# Al llegar las rebajas,  durante el primer mes realiza un 30% descuento sobre el precio inicial 
# y en el segundo mes un 40 % también sobre el precio inicial punto sabiendo que vende un total de
# 600 pares de calcetines por 597600 pts  y que en las rebajas ha vendido la mitad de dicho total (de calcetines), 
# ¿a cuántos pares de calcetines se le aplica se le ha aplicado un descuento del 40 %?""",
#              [Eq(x+y+z,600),Eq(1200*x+0.7*1200*y+0.6*1200*z,597600),Eq(y+z,300)],
# #              [Eq(x+y+z,600),Eq(10*x+7*y+6*z,4980),Eq(y+z,300)]
#             ],
#             ["""Una persona trata de adivinar, mediante ciertas pistas,  el coste de tre productos A, B y C  que un
# amigo suyo ha comprado:  pista 1:  si compro una unidad de A,  2 de B  y una de C  me gasto 900 pesetas. Pista 2:  si
# compro k unidades de A, k+3 de y 3 de C me gasto 2950 pts. a) ¿Hay algún valor de k para el cual estas dos pistas no 
# son compatibles? b)  en la pista 2 se toma igual k=4, ¿ es posible saber el coste de cada uno de los productos? c) el 
# amigo le dice finalmente que el producto se vale cinco veces lo que vale el producto A y que en la pista 2 se tiene K=4. 
# ¿Cuánto valen A, B y C?"""],
#             [Eq(5*x-5*y-z,0),Eq(2*x-y-z,6),Eq(x-3*y+2*z,8)],
#             ["""La suma de las tres cifras de un número es 16, y la suma de la primera y la tercera igual 
# a la segunda. Permutando entre si dichas cifras ( primera y tercera) resulta un número que supera en 198 
# unidades al número dado.¿Cuál es dicho número?""",
#             [Eq(x+y+z,16), Eq(x+z,y),Eq(100*z+10*y+x-198,100*x+10*y+z)]],
#             [r"""Una empresa envasadora ha comprado un total de 1500 cajas de pescado en tres mercados 
# diferentes, a un precio por caja 30, 20 y 40  euros respectivamente. el coste total de la operación ha sido 
# de 40500\euro. Calcular cuánto ha pagado la empresa en cada mercado, sabiendo que en el primero de ellos ha 
# comprado el 30\% de las cajas """,
#             [Eq(x+y+z,1500), Eq(30*x+20*y+40*z,40500),Eq(x,450)]],
              [r"""Un mayorista de café dispone de tres tipos base:  Moka, Brasil y Colombia, para preparar 3 tipos de mezcla: A, B y C, envasa en sacos de 60 kg con los siguientes contenidos en kilos y precio:
\begin{tabular}{|c|c|c|c|}
\hline
& Mezcla A & Mezcla B & Mezcla C \\
\hline
Moka & 15 & 30 & 12 \\
\hline
Brasil & 20 & 10 & 18 \\
\hline
Colombia & 15 & 20 & 30 \\
\hline
Precio/kg & 4 & 4.5 & 4.7 \\
\hline
\end{tabular}

Suponiendo que el preparado de las mezclas no supone coste alguno, ¿cuál es el precio de cada uno de los tipos base de café?
""",
#               [Eq(15*x+30*y+15*z,240), Eq(30*x+10*y+20*z,270),Eq(12*x+18*y+30*z,282)]],
                [Eq(x+2*y+z,16), Eq(3*x+y+2*z,27),Eq(2*x+3*y+5*z,47)]],
           ]


for ej in problemas :
    print(latex(ej[0]))
    print(r"El sistema a resolver es:"+sistema_a_latex(ej[1])+r" \\ ")
    print(discusion_solucion([nsimplify(i) for i in ej[1]]))

Un mayorista de café dispone de tres tipos base:  Moka, Brasil y Colombia, para preparar 3 tipos de mezcla: A, B y C, envasa en sacos de 60 kg con los siguientes contenidos en kilos y precio:
\begin{tabular}{|c|c|c|c|}
\hline
& Mezcla A & Mezcla B & Mezcla C \\
\hline
Moka & 15 & 30 & 12 \\
\hline
Brasil & 20 & 10 & 18 \\
\hline
Colombia & 15 & 20 & 30 \\
\hline
Precio/kg & 4 & 4.5 & 4.7 \\
\hline
\end{tabular}

Suponiendo que el preparado de las mezclas no supone coste alguno, ¿cuál es el precio de cada uno de los tipos base de café?

El sistema a resolver es:$\left\{ \begin{matrix}x + 2 y + z = 16 \\ 3 x + y + 2 z = 27 \\ 2 x + 3 y + 5 z = 47 \\ \end{matrix}\right.$ \\ 
Discusión y resolución por Gauss:
⎡1  2    1     16 ⎤
⎢                 ⎥
⎢0  -5   -1   -21 ⎥
⎢                 ⎥
⎣0  0   16/5  96/5⎦
det(A)=-16
[16/5, 96/5]
S.C.D.
[4, 3, 6]
[(x, 4, [1  2  1  16]), (y, 3, [0  -5  -1  -21]), (z, 6, [0  0  16/5  96/5])]
Matrix([[0, 0, 16/5, 96/5]]) --> z = 6
Matrix([[0, -5, -1, -21]]) 

In [6]:
# Ejercicio de tuberias selectividad

sist1=[Eq(y+z,40),Eq(x,y+10)]
print(sistema_a_latex(sist1))    
A, b = linear_eq_to_matrix(sist1,[x,y,z])
AA = A.row_join(b)
AAs = AA.LUdecomposition()[1].applyfunc(nsimplify)
print(latex(A))
print(latex(AA))
print(latex(AAs))

sist2=[Eq(y+z,40),Eq(x,y+10),Eq(y-3*z,0)]
print(sistema_a_latex(sist2)) 
print(discusion_solucion(sist2))




$\left\{ \begin{matrix}y + z = 40 \\ x = y + 10 \\ \end{matrix}\right.$
\left[\begin{matrix}0 & 1 & 1\\1 & -1 & 0\end{matrix}\right]
\left[\begin{matrix}0 & 1 & 1 & 40\\1 & -1 & 0 & 10\end{matrix}\right]
\left[\begin{matrix}1 & -1 & 0 & 10\\0 & 1 & 1 & 40\end{matrix}\right]
$\left\{ \begin{matrix}y + z = 40 \\ x = y + 10 \\ y - 3 z = 0 \\ \end{matrix}\right.$
Discusión y resolución por Gauss:
⎡1  -1  0   10 ⎤
⎢              ⎥
⎢0  1   1   40 ⎥
⎢              ⎥
⎣0  0   -4  -40⎦
det(A)=4
[-4, -40]
S.C.D.
[40, 30, 10]
[(x, 40, [1  -1  0  10]), (y, 30, [0  1  1  40]), (z, 10, [0  0  -4  -40])]
Matrix([[0, 0, -4, -40]]) --> z = 10
Matrix([[0, 1, 1, 40]]) --> y = 30
Matrix([[1, -1, 0, 10]]) --> x = 40
Por rangos y determinantes:
Por Cramer:
Delta_i, A.det, solucion_i : 
⎡   ⎡40  1   1 ⎤            ⎤
⎢   ⎢          ⎥            ⎥
⎢x, ⎢10  -1  0 ⎥, 160, 4, 40⎥
⎢   ⎢          ⎥            ⎥
⎣   ⎣0   1   -3⎦            ⎦
Delta_i, A.det, solucion_i : 
⎡   ⎡0  40  1 ⎤            ⎤
⎢   ⎢         ⎥  

In [7]:
sist1=[Eq(x+3*y+7*z,565),Eq(x+4*y+10*z,740),Eq(x+y+z,k)]

print(sistema_a_latex(sist1))    
A, b = linear_eq_to_matrix(sist1,[x,y,z])
AA = A.row_join(b)
AAs = AA.LUdecomposition()[1].applyfunc(nsimplify)
print(latex(A))
print(latex(AA))
print(latex(AAs))


$\left\{ \begin{matrix}x + 3 y + 7 z = 565 \\ x + 4 y + 10 z = 740 \\ x + y + z = k \\ \end{matrix}\right.$
\left[\begin{matrix}1 & 3 & 7\\1 & 4 & 10\\1 & 1 & 1\end{matrix}\right]
\left[\begin{matrix}1 & 3 & 7 & 565\\1 & 4 & 10 & 740\\1 & 1 & 1 & k\end{matrix}\right]
\left[\begin{matrix}1 & 3 & 7 & 565\\0 & 1 & 3 & 175\\0 & 0 & 0 & k - 215\end{matrix}\right]


In [8]:
# Varios amigos pegan en un bar 755 pesetas por 5 cervezas tres bocadillos y dos cafés. Al día siguiente consumen 3 cervezas dos bocadillos y 4 cafés por lo que pagan 645 pesetas
# Si al tercer día consumen 7 cervezas y 4 bocadillos, ¿qué precio deberían pagar por ello?
# ¿Puede saberse de los datos anteriores el precio de una cerveza o un bocadillo o un café? Si además sabemos que un café vale 60 pesetas ¿puede saberse el precio de una cerveza o un bocadillo?


# sist1 = [Eq(5*x+3*y+2*z,755),Eq(3*x+2*y+4*z,645),Eq(7*x+4*y,k)]
sist1 = [Eq(5*x+3*y+2*z,755),Eq(3*x+2*y+4*z,645),Eq(z,60)]
print(sistema_a_latex(sist1))    
A, b = linear_eq_to_matrix(sist1,[x,y,z])
AA = A.row_join(b)
AAs = AA.LUdecomposition()[1].applyfunc(nsimplify)
print(latex(A))
print(latex(AA))
print(latex(AAs))


# print(latex(A))
# A.det()
print(discusion_solucion(sist1))

$\left\{ \begin{matrix}5 x + 3 y + 2 z = 755 \\ 3 x + 2 y + 4 z = 645 \\ z = 60 \\ \end{matrix}\right.$
\left[\begin{matrix}5 & 3 & 2\\3 & 2 & 4\\0 & 0 & 1\end{matrix}\right]
\left[\begin{matrix}5 & 3 & 2 & 755\\3 & 2 & 4 & 645\\0 & 0 & 1 & 60\end{matrix}\right]
\left[\begin{matrix}5 & 3 & 2 & 755\\0 & \frac{1}{5} & \frac{14}{5} & 192\\0 & 0 & 1 & 60\end{matrix}\right]
Discusión y resolución por Gauss:
⎡5   3    2    755⎤
⎢                 ⎥
⎢0  1/5  14/5  192⎥
⎢                 ⎥
⎣0   0    1    60 ⎦
det(A)=1
[1, 60]
S.C.D.
[55, 120, 60]
[(x, 55, [5  3  2  755]), (y, 120, [0  1/5  14/5  192]), (z, 60, [0  0  1  60]
)]
Matrix([[0, 0, 1, 60]]) --> z = 60
Matrix([[0, 1/5, 14/5, 192]]) --> y = 120
Matrix([[5, 3, 2, 755]]) --> x = 55
Por rangos y determinantes:
Por Cramer:
Delta_i, A.det, solucion_i : 
⎡   ⎡755  3  2⎤           ⎤
⎢   ⎢         ⎥           ⎥
⎢x, ⎢645  2  4⎥, 55, 1, 55⎥
⎢   ⎢         ⎥           ⎥
⎣   ⎣60   0  1⎦           ⎦
Delta_i, A.det, solucion_i : 
⎡   ⎡5  755  2⎤      

In [9]:
# En una excavación arqueológica se han encontrado sortijas monedas y pendientes.  
# Una sortija, una moneda y un pendiente pesan conjuntamente 30 gramos. Además, cuatro sortijas
# tres monedas y dos pendientes han dado un peso total de 90 gramos. El peso de un objeto deformado 
# irreconocible es de 18 gramos determina si el mencionado objeto es una sortija una moneda o un 
# pendiente sabiendo que los objetos que son del mismo tipo pesan lo mismo

sistemas = [[Eq(x+y+z,30),Eq(4*x+3*y+2*z,90),Eq(x,18)],[Eq(x+y+z,30),Eq(4*x+3*y+2*z,90),Eq(y,18)],[Eq(x+y+z,30),Eq(4*x+3*y+2*z,90),Eq(z,18)]]
for sis in sistemas :
    print(sistema_a_latex(sis))
    print(discusion_solucion(sis))

$\left\{ \begin{matrix}x + y + z = 30 \\ 4 x + 3 y + 2 z = 90 \\ x = 18 \\ \end{matrix}\right.$
Discusión y resolución por Gauss:
⎡1  1   1   30 ⎤
⎢              ⎥
⎢0  -1  -2  -30⎥
⎢              ⎥
⎣0  0   1   18 ⎦
det(A)=-1
[1, 18]
S.C.D.
[18, -6, 18]
[(x, 18, [1  1  1  30]), (y, -6, [0  -1  -2  -30]), (z, 18, [0  0  1  18])]
Matrix([[0, 0, 1, 18]]) --> z = 18
Matrix([[0, -1, -2, -30]]) --> y = -6
Matrix([[1, 1, 1, 30]]) --> x = 18
Por rangos y determinantes:
Por Cramer:
Delta_i, A.det, solucion_i : 
⎡   ⎡30  1  1⎤             ⎤
⎢   ⎢        ⎥             ⎥
⎢x, ⎢90  3  2⎥, -18, -1, 18⎥
⎢   ⎢        ⎥             ⎥
⎣   ⎣18  0  0⎦             ⎦
Delta_i, A.det, solucion_i : 
⎡   ⎡1  30  1⎤           ⎤
⎢   ⎢        ⎥           ⎥
⎢y, ⎢4  90  2⎥, 6, -1, -6⎥
⎢   ⎢        ⎥           ⎥
⎣   ⎣1  18  0⎦           ⎦
Delta_i, A.det, solucion_i : 
⎡   ⎡1  1  30⎤             ⎤
⎢   ⎢        ⎥             ⎥
⎢z, ⎢4  3  90⎥, -18, -1, 18⎥
⎢   ⎢        ⎥             ⎥
⎣   ⎣1  0  18⎦             ⎦
\textbf{

In [10]:
# Un cajero automático contiene solo billetes de 10, 20 y 50 \euro. 
# En total hay 130 billetes con un importe de 3000 \euro.
# ¿Es posible que en el cajero haya el triple de número de billetes de 10 que de 50?
# Suponiendo que el número de billetes de 10 es el doble que el número de billetes de 50, 
# calcula cuántos billetes hay de cada tipo.

sistemas = [[Eq(x+y+z,130),Eq(10*x+20*y+50*z,3000),Eq(x,3*z)],[Eq(x+y+z,130),Eq(10*x+20*y+50*z,3000),Eq(x,2*z)]]
for sis in sistemas :
    print(sistema_a_latex(sis))
    print(discusion_solucion(sis))


$\left\{ \begin{matrix}x + y + z = 130 \\ 10 x + 20 y + 50 z = 3000 \\ x = 3 z \\ \end{matrix}\right.$
Como $rg(A)=2 < rg(A^*)=3 \to$ S.I. \\ Ya que escalonando la matriz ampliada: \\ $A^*= \left(\begin{matrix}1 & 1 & 1 & 130\\10 & 20 & 50 & 3000\\1 & 0 & -3 & 0\end{matrix}\right) \thicksim \left(\begin{matrix}1 & 1 & 1 & 130\\0 & 10 & 40 & 1700\\0 & 0 & 0 & 40\end{matrix}\right)$ \\$\left|\begin{matrix}1 & 1 & 1\\0 & 10 & 40\\0 & 0 & 0\end{matrix}\right|=0 \quad y \quad \left|\begin{matrix}1 & 1 & 130\\10 & 40 & 1700\\0 & 0 & 40\end{matrix}\right|=1200$
$\left\{ \begin{matrix}x + y + z = 130 \\ 10 x + 20 y + 50 z = 3000 \\ x = 2 z \\ \end{matrix}\right.$
Discusión y resolución por Gauss:
⎡1  1   1   130 ⎤
⎢               ⎥
⎢0  10  40  1700⎥
⎢               ⎥
⎣0  0   1    40 ⎦
det(A)=10
[1, 40]
S.C.D.
[80, 10, 40]
[(x, 80, [1  1  1  130]), (y, 10, [0  10  40  1700]), (z, 40, [0  0  1  40])]
Matrix([[0, 0, 1, 40]]) --> z = 40
Matrix([[0, 10, 40, 1700]]) --> y = 10
Matrix([[1, 1, 1, 130]

In [11]:
sistemas = [
#             [Eq(-3*x+2*y+3*z,-2),Eq(2*x+k*y-5*z,-4),Eq(x+y+2*z,2)],
#            [Eq(k*x + y + z , k),Eq(x + k*y + z , k), Eq(x + y + k*z , k)],
#             [Eq(x + y + z , k + 2),Eq(x - k*y + z , 1),Eq(k*x + y + z , 4)],
#             [Eq(k*x+(k**2+1)*y+k*z,k),Eq(x+k*y+z,0),Eq(x+(k+1)*y+k**2*z,2*k-1)],
#             [Eq(2*x - 5*y + 3*z , 0),Eq(x - y + z , 0),Eq(3*x + k*y + z , 0)],
#             [Eq(x + k*y - z , 0),Eq(12*x - 3*y - 2*z , 0),Eq(x - 2*y + z , 0)],
#             [Eq(x + y + 2*z , 0),Eq(k*x + y - z , k - 2), Eq(3*x + k*y + z , k - 2)],
            #[Eq(k*x+y,1),Eq(2*x+(k+1)*y,2),Eq(x+k*y,1)] son linealmente dependientes las filas de A ,
            [Eq(y+k*z,1),Eq(k*x-y+z,1),Eq(y-z,-k)]
            ]

texto_ejercicio = r'Ejercicios de sistemas con un parámetro:'

n_ejercicio = ''

display(md(""+texto_ejercicio+""))


enunciado_latex, enunciado, solucion = [],[],[]
for s in sistemas :
    enl = sistema_a_latex(s)
    enunciado_latex.append(enl)
    en = str(s)
    enunciado.append(en)
    sol=discusion_solucion(s)
    solucion.append(sol)
    
df_ejercicios = df_ejercicios.append(añadir_ejercicios(enunciado_latex, enunciado, solucion, texto_ejercicio, curso, titulo, n_ejercicio, n_columnas=1, enun_tex=False, sol_tex=False), ignore_index=True)

Ejercicios de sistemas con un parámetro:

Discusión y resolución por Gauss:
⎡k  -1    1       1   ⎤
⎢                     ⎥
⎢0  1     k       1   ⎥
⎢                     ⎥
⎣0  0   -k - 1  -k - 1⎦
det(A)=k**2 + k
[-k - 1, -k - 1]
-1
Si k = -1 -->  0z=0 --> S.C.I
[y - z = 1, -x - y + z = 1, y - z = 1]
[(x, -2, [-1  -1  1  1]), (y, z + 1, [0  1  -1  1]), (z, z, [0  0  0  0])]
Matrix([[0, 0, 0, 0]]) --> z = z
Matrix([[0, 1, -1, 1]]) --> y = z + 1
Matrix([[-1, -1, 1, 1]]) --> x = -2
si k <> [-1]  --> S.C.D.
⎡1 - k          ⎤
⎢─────, 1 - k, 1⎥
⎣  k            ⎦
⎡⎛   1 - k               ⎞                                                    
⎢⎜x, ─────, [k  -1  1  1]⎟, (y, 1 - k, [0  1  k  1]), (z, 1, [0  0  -k - 1  -k
⎣⎝     k                 ⎠                                                    

      ⎤
 - 1])⎥
      ⎦
Matrix([[0, 0, -k - 1, -k - 1]]) --> z = 1
Matrix([[0, 1, k, 1]]) --> y = 1 - k
Matrix([[k, -1, 1, 1]]) --> x = (1 - k)/k
Por rangos y determinantes:
⎡0   1   -1⎤
⎢          ⎥
⎢-1  -1  1 ⎥
⎢          ⎥
⎣0   1   -1⎦
2
2

In [12]:
df_ejercicios.to_csv('ejercicios_sistemas_param.csv', index = False)

In [13]:
# df_ejercicios = pd.read_csv('ejercicios_sistemas_param.csv', index_col= None)

In [14]:
df_ejercicios

,enunciado_latex,enunciado,solucion,texto,curso,titulo,n_ejercicio,dificultad,n_columnas,puntos,enun_tex,sol_tex
0,$\left\{ \begin{matrix}k z + y = 1 \\ k x - y ...,"[Eq(k*z + y, 1), Eq(k*x - y + z, 1), Eq(y - z,...",\textbf{Discusión y resolución por Gauss:} Esc...,Ejercicios de sistemas con un parámetro:,2BC,Ejercicios de sistemas,,1,1,1,False,False


In [15]:
escribir_preambulo(fichero, titulo, 'ejercicios')
for s in df_ejercicios.groupby('n_ejercicio').count().index : 
    display(md("**Ejercicio: **" + s ))
    display(df_ejercicios[df_ejercicios.n_ejercicio == s])
    escribir_ejercicios(df_ejercicios[df_ejercicios.n_ejercicio == s],fichero)

escribir_fin(fichero)

**Ejercicio: **

,enunciado_latex,enunciado,solucion,texto,curso,titulo,n_ejercicio,dificultad,n_columnas,puntos,enun_tex,sol_tex
0,$\left\{ \begin{matrix}k z + y = 1 \\ k x - y ...,"[Eq(k*z + y, 1), Eq(k*x - y + z, 1), Eq(y - z,...",\textbf{Discusión y resolución por Gauss:} Esc...,Ejercicios de sistemas con un parámetro:,2BC,Ejercicios de sistemas,,1,1,1,False,False
